In this notebook I will plot all the longidutinal data for each individual separately

In [1]:
import src.data.breathe_data as br
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import src.data.helpers as dh
from datetime import timedelta
import src.o2_fev1_analysis.smooth as smooth
import numpy as np

In [2]:
# df = br.build_O2_FEV1_FEF2575_df(meas_file=2, remove_nan=False)
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")
df["PEF (L/s)"] = df.PEF / 60

In [3]:
# df.to_excel(f"{dh.get_path_to_main()}ExcelFiles/BR/BR_O2_FEV1_FEF2575_PEF_Nan.xlsx", index=False)

In [4]:
def apply_new_smoothing(df, col, scale, shift):
    df[f"ec{col}_2"] = df[col]
    no_nan_mask = ~df[f"ec{col}_2"].isna()
    df.iloc[no_nan_mask] = smooth.identify_and_replace_outliers_up(
        df[no_nan_mask], f"ec{col}_2", scale=scale, shift=shift
    ).copy()
    df[f"ec{col}_3"] = smooth.smooth_vector(df[f"ec{col}_2"].to_numpy(), "max")

    return df


df1 = (
    df.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEV1", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "FEF2575", 3, 0.5))
    .reset_index(drop=True)
)
df1 = (
    df1.groupby(by="ID")
    .apply(lambda x: apply_new_smoothing(x, "PEF (L/s)", 3, 1))
    .reset_index(drop=True)
)


def plot_profile_for_ID(df_for_ID):
    df_for_ID = df_for_ID.reset_index()
    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.04,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["O2 Saturation"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=2, col=1)
    fig.add_trace(
        go.Scatter(y=df_for_ID.ecFEV1_3, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.FEF2575,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=3,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=3, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.ecFEF2575_3, x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["PEF (L/s)"],
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=4,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=4, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["ecPEF (L/s)_3"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=4,
        col=1,
    )
    fig.update_yaxes(
        title="SpO2<br>(%)",
        row=1,
        col=1,
        range=[
            np.nanmin(df["O2 Saturation"]) * 0.98,
            np.nanmax(df["O2 Saturation"]) * 1.02,
        ],
    )
    fig.update_yaxes(
        title="FEV1<br>(L)",
        row=2,
        col=1,
        range=[np.nanmin(df.FEV1) * 0.98, np.nanmax(df.FEV1) * 1.02],
    )
    fig.update_yaxes(
        title="FEF2575<br>(L/s)",
        row=3,
        col=1,
        range=[np.nanmin(df.FEF2575) * 0.98, np.nanmax(df.FEF2575) * 1.02],
    )
    fig.update_yaxes(
        title="PEF<br>(L/s)",
        row=4,
        col=1,
        range=[np.nanmin(df.PEF) / 60 * 0.98, np.nanmax(df.PEF) / 60 * 1.02],
    )
    fig.update_yaxes(nticks=10)
    fig.update_traces(marker=dict(size=3))
    title = f"Measures for ID {df_for_ID.ID[0]}"
    fig.update_layout(
        font=dict(size=8), showlegend=False, title=title, height=1000, width=1400
    )
    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/ID_profiles/{title}.pdf")
    return


df1.groupby(by="ID").apply(plot_profile_for_ID)

ID 101 - Outlier up for ecFEV1_2, day 2023-10-24: 2.52 > 2.19 and > 2.22 (mean=1.72,std=0.16), update to 1.64
ID 105 - Outlier up for ecFEV1_2, day 2019-01-18: 2.19 > 1.76 and > 2.08 (mean=1.58,std=0.06), update to 2.01
ID 107 - Outlier up for ecFEV1_2, day 2023-09-19: 2.71 > 2.32 and > 2.29 (mean=1.78,std=0.18), update to 1.66
ID 123 - Outlier up for ecFEV1_2, day 2019-09-18: 3.86 > 3.76 and > 3.72 (mean=3.22,std=0.18), update to 3.21
ID 146 - Outlier up for ecFEV1_2, day 2019-10-26: 1.56 > 1.23 and > 1.53 (mean=1.03,std=0.07), update to 2.24
ID 221 - Outlier up for ecFEV1_2, day 2022-08-09: 3.03 > 2.45 and > 2.44 (mean=1.94,std=0.17), update to 2.06
ID 244 - Outlier up for ecFEV1_2, day 2022-08-27: 4.38 > 3.86 and > 3.61 (mean=3.11,std=0.25), update to 3.06
ID 104 - Outlier up for ecFEF2575_2, day 2023-07-26: 1.73 > 1.72 and > 1.67 (mean=1.17,std=0.18), update to 1.12
ID 105 - Outlier up for ecFEF2575_2, day 2019-01-18: 2.48 > 1.81 and > 1.99 (mean=1.49,std=0.11), update to 2.47
ID 1

""


In [7]:
df1

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,ecFEF2575,ecPEF,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEV1_2,ecFEF2575_2,ecPEF_2
0,101,2019-01-25,1.31,97.0,0.54,NaN,1.31,0.67,NaN,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,99.845492,1.31,0.54,NaN
1,101,2019-01-26,1.31,98.0,0.57,NaN,1.31,0.67,NaN,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,100.874827,1.31,0.57,NaN
2,101,2019-01-27,1.31,96.0,0.67,NaN,1.31,0.69,NaN,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,98.816157,1.31,0.67,NaN
3,101,2019-01-28,1.30,96.0,0.69,NaN,1.31,0.69,NaN,53,Male,173.0,3.610061,97.150104,36.287474,36.010470,98.816157,1.30,0.69,NaN
4,101,2019-01-29,1.28,98.0,0.60,NaN,1.30,0.69,NaN,53,Male,173.0,3.610061,97.150104,36.010470,35.456463,100.874827,1.28,0.60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,123,2022-07-21,3.84,98.0,4.47,486.0,3.84,4.54,555.0,20,Male,175.0,4.489669,97.114230,85.529684,85.529684,100.912090,3.84,4.47,486.0
9996,123,2022-07-22,3.80,98.0,4.54,555.0,3.85,4.70,555.0,20,Male,175.0,4.489669,97.114230,85.752418,84.638750,100.912090,3.80,4.54,555.0
9997,123,2022-07-24,3.85,98.0,4.70,547.0,3.87,4.73,571.0,20,Male,175.0,4.489669,97.114230,86.197885,85.752418,100.912090,3.85,4.70,547.0
9998,123,2022-07-25,3.87,98.0,4.73,571.0,3.91,4.73,571.0,20,Male,175.0,4.489669,97.114230,87.088819,86.197885,100.912090,3.87,4.73,571.0


In [19]:
def plot_profile_for_ID(df_for_ID):
    df_for_ID = df_for_ID.reset_index()
    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.04,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["O2 Saturation"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=2, col=1)
    fig.add_trace(
        go.Scatter(y=df_for_ID.ecFEV1_2, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.FEF2575,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=3,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=3, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.ecFEF2575_2, x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.PEF / 60,
            x=df_for_ID["Date Recorded"],
            mode="markers",
        ),
        row=4,
        col=1,
    )
    fig.update_traces(marker=dict(color="yellow"), row=4, col=1)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID.ecPEF_2 / 60, x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=4,
        col=1,
    )
    fig.update_yaxes(
        title="SpO2<br>(%)",
        row=1,
        col=1,
        range=[
            np.nanmin(df["O2 Saturation"]) * 0.98,
            np.nanmax(df["O2 Saturation"]) * 1.02,
        ],
    )
    fig.update_yaxes(
        title="FEV1<br>(L)",
        row=2,
        col=1,
        range=[np.nanmin(df.FEV1) * 0.98, np.nanmax(df.FEV1) * 1.02],
    )
    fig.update_yaxes(
        title="FEF2575<br>(L/s)",
        row=3,
        col=1,
        range=[np.nanmin(df.FEF2575) * 0.98, np.nanmax(df.FEF2575) * 1.02],
    )
    fig.update_yaxes(
        title="PEF<br>(L/s)",
        row=4,
        col=1,
        range=[np.nanmin(df.PEF) / 60 * 0.98, np.nanmax(df.PEF) / 60 * 1.02],
    )
    fig.update_yaxes(nticks=10)
    fig.update_traces(marker=dict(size=3))
    title = f"Measures for ID {df_for_ID.ID[0]}"
    fig.update_layout(
        font=dict(size=8), showlegend=False, title=title, height=1000, width=1400
    )
    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/ID_profiles/{title}.pdf")
    return -1

In [20]:
df1[0:10000].groupby(by="ID").apply(plot_profile_for_ID)

ID
101   -1
102   -1
103   -1
104   -1
105   -1
106   -1
107   -1
108   -1
109   -1
110   -1
111   -1
112   -1
113   -1
114   -1
115   -1
116   -1
117   -1
118   -1
119   -1
120   -1
121   -1
122   -1
123   -1
dtype: int64